In [1]:
with open("data.txt" , 'r',encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters is :: ", len(text))

length of dataset in characters is ::  1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [14]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda s: ''.join([itos[c] for c in s])

In [17]:
encode("hii there")

[46, 47, 47, 1, 58, 46, 43, 56, 43]

In [18]:
print(decode([46, 47, 47, 1, 58, 46, 43, 56, 43]))

['h', 'i', 'i', ' ', 't', 'h', 'e', 'r', 'e']
